In [1]:
import pandas as pd
from pathlib import Path

RAW_DIR = Path("../data/raw")
PROCESSED_DIR = Path("../data/processed")
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)


In [2]:
events = pd.read_csv(RAW_DIR / "events_epl_2015_16.csv")
matches = pd.read_csv(RAW_DIR / "matches_epl_2015_16.csv")
competitions = pd.read_csv(RAW_DIR / "competitions.csv")


C:\Users\Βασιλης\AppData\Local\Temp\ipykernel_16208\3364195361.py:1: DtypeWarning: Columns (9,11,16,17,18,19,22,27,36,44,45,51,75,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117) have mixed types. Specify dtype option on import or set low_memory=False.
  events = pd.read_csv(RAW_DIR / "events_epl_2015_16.csv")


In [3]:
raw_events = events.copy()

In [4]:
raw_matches = matches.copy()

Lets create the team dimention

In [9]:
matches.columns


Index(['match_id', 'match_date', 'kick_off', 'competition', 'season',
       'home_team', 'away_team', 'home_score', 'away_score', 'match_status',
       'match_status_360', 'last_updated', 'last_updated_360', 'match_week',
       'competition_stage', 'stadium', 'referee', 'home_managers',
       'away_managers', 'data_version', 'shot_fidelity_version',
       'xy_fidelity_version'],
      dtype='object')

In [10]:
dim_team = (
    events[["team_id", "team"]]
    .dropna()
    .drop_duplicates()
    .rename(columns={"team": "team_name"})
    .reset_index(drop=True)
)

dim_team.shape


(20, 2)

In [11]:
dim_team.to_csv(PROCESSED_DIR / "dim_team.csv", index=False)


Lets create the player dimention

In [24]:
players_raw = events[["player_id", "player", "position"]].dropna(subset=["player_id"])
players_raw.shape


(1309046, 3)

In [25]:
players_raw.head()

,player_id,player,position
6,3343.0,Dan Gosling,Right Center Midfield
7,3346.0,Joshua King,Center Forward
8,3344.0,Andrew Surman,Center Defensive Midfield
9,6409.0,Adam Smith,Right Back
10,3608.0,Simon Francis,Right Center Back


Players have many positions, i choose the most common one.

In [26]:
players_raw = players_raw.rename(columns={"player": "player_name", "position": "position_name"})

primary_pos = (
    players_raw.groupby(["player_id", "player_name"])["position_name"]
    .agg(lambda s: s.value_counts().idxmax())
    .reset_index()
)

dim_player = primary_pos
dim_player.shape


(549, 3)

In [27]:
dim_player.to_csv(PROCESSED_DIR / "dim_player.csv", index=False)


dimention match

In [28]:
dim_match = matches[
    [
        "match_id",
        "match_date",
        "kick_off",
        "home_team",
        "away_team",
        "home_score",
        "away_score",
        "match_week"
    ]
].copy()


In [29]:
dim_match = dim_match.rename(
    columns={
        "home_team": "home_team_name",
        "away_team": "away_team_name"
    }
)


In [31]:
dim_match.to_csv(PROCESSED_DIR / "dim_match.csv", index=False)


competition dimention

In [34]:
dim_competition = competitions[
    competitions["competition_name"] == "Premier League"
][["competition_id", "competition_name", "country_name"]].drop_duplicates()


In [36]:
dim_competition.to_csv(PROCESSED_DIR / "dim_competition.csv", index=False)


Dim season

In [39]:
dim_season = competitions[
    (competitions["competition_name"] == "Premier League")
][["season_id", "season_name"]].drop_duplicates()


In [40]:
dim_season.to_csv(PROCESSED_DIR / "dim_season.csv", index=False)
